In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 15 14:49:58 2022

@author: Sidi Wu and Cédric Beaulac

Functional autoencoder implementation
"""

# Import modules
import torch
import torch.nn.init as init
import torch.nn.functional as F
import torch.nn as nn
import Hyper_parameter_search
from Hyper_parameter_search import weights_init_normal
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pandas as pd
import numpy as np
from numpy import *
import seaborn as sns
import matplotlib
#matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
# import skfda as fda
# from skfda import representation as representation
# from skfda.exploratory.visualization import FPCAPlot
# # from skfda.exploratory.visualization import FPCAPlot
# # from skfda.preprocessing.dim_reduction import FPCA
# # from skfda.representation.basis import BSpline, Fourier, Monomial
import scipy
from scipy.interpolate import BSpline
import os
import ignite
import sklearn
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
import random
from random import seed
from scipy import stats
import statistics
from statistics import stdev
import skfda

import skfda
from skfda import FDataGrid as fd
from skfda.representation.basis import BSpline as B
torch.manual_seed(123)


In [3]:
Phoneme=skfda.datasets.fetch_cran("phoneme",package_name="fda.usc")
phoneme=Phoneme['phoneme']

x_train,x_test,y_train,y_test=phoneme['learn'],phoneme['test'],phoneme['classlearn'],phoneme['classtest']
x_train.shape,x_test.shape,y_train.categories,y_test.categories
##Phonèmes enregistrés à classer dans 5 catégories

c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:321: UserWarning: Unknown encoding. Assumed ASCII.
  warnings.warn("Unknown encoding. Assumed ASCII.")
c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:321: UserWarning: Unknown encoding. Assumed ASCII.
  warnings.warn("Unknown encoding. Assumed ASCII.")


((250,),
 (250,),
 Index(['1', '2', '3', '4', '5'], dtype='object'),
 Index(['1', '2', '3', '4', '5'], dtype='object'))

In [4]:

x_train_tensor=torch.tensor(x_train.data_matrix).reshape(250,1,150).cuda().float()
x_test_tensor=torch.tensor(x_test.data_matrix).reshape(250,1,150).cuda().float()
y_train_tensor=torch.tensor(y_train.codes).unsqueeze(1).unsqueeze(2).long().cuda()
y_test_tensor=torch.tensor(y_test.codes).unsqueeze(1).unsqueeze(2).long().cuda()

X=torch.cat([x_train_tensor,x_test_tensor])
Y=torch.cat([y_train_tensor,y_test_tensor])


In [5]:
basis=B(knots=linspace(1,150,6),order=4)
n_basis=basis.n_basis


In [6]:
output_size=5

class Project_classifier(nn.Module):
    def __init__(self,basis):
        super(Project_classifier,self).__init__()
        self.basis=basis
        self.fc_block=nn.Sequential(
            nn.Linear(self.basis.n_basis,64),
            nn.LeakyReLU(negative_slope=0.17),
            nn.BatchNorm1d(64),
            nn.Linear(64,32),
            nn.Tanh(),
            nn.LeakyReLU(negative_slope=0.17),
            nn.BatchNorm1d(32),
            nn.Tanh(),
            nn.Linear(32,16),
            nn.LeakyReLU(negative_slope=0.17),
            nn.Linear(16,output_size),
            
        )
        
    def Project(self, x, basis_fc):
        # basis_fc: n_time X nbasis
        # x: n_subject X n_time
        # w = x.size(1)-1
        # W = torch.tensor([1/(2*w)]+[1/w]*(w-1)+[1/(2*w)])
        
        f = torch.matmul(torch.tensor(x.data_matrix[:,:,0]).float().cuda(), torch.t(basis_fc))
        return f
    def forward(self,x):
        basis_fc=self.basis(x.grid_points[0],derivative=0)[:,:,0]
        basis_fc=torch.tensor(basis_fc).float().cuda()
        proj_out=self.Project(x,basis_fc=basis_fc)
        lin_out=self.fc_block(proj_out)
        return lin_out.float().unsqueeze(2).unsqueeze(3)
    

In [7]:
classifier=Project_classifier(basis=basis).cuda().apply(weights_init_normal)
basis_fc=basis_fc=basis(x_train.grid_points[0],derivative=0)[:,:,0]
basis_fc=torch.tensor(basis_fc).float().cuda()

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_3556\1916630021.py:2: DeprecationWarning: The method 'evaluate' is deprecated. Please use the normal calling notation on the basis object instead.
  basis_fc=basis_fc=basis.evaluate(x_train.grid_points[0],derivative=0)[:,:,0]


In [8]:
lr=0.0001
betas=[0.5,0.999]
batch_size=36
loss=nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=lr, betas=betas)

In [9]:
from tqdm import tqdm


def train(n_epochs=50, optimizer=optimizer, loss=loss, batch_size=batch_size):
    for epoch in tqdm(range(n_epochs)):
        train_loss = torch.tensor(0).cuda().long()
        
        # Mélanger les données d'entraînement
        indices = list(range(len(x_train)))
        random.shuffle(indices)
        
        batch_index = 0  # Indice de batch
        
        for i in range(int(len(x_train) / batch_size)):
            # Obtenir les indices des données mélangées
            batch_indices = indices[batch_index:batch_index+batch_size]
            
            functions_train = x_train[batch_indices]
            labels_train = y_train_tensor[batch_indices]
            
            optimizer.zero_grad()
            output = classifier(functions_train)
            loss_value = loss(input=output, target=labels_train)
            
            loss_value.backward()
            optimizer.step()
            train_loss += loss_value.long()
            
            batch_index += batch_size  # Passer au prochain batch
            
        return train_loss, loss_value

In [133]:
eval_points=np.linspace(1,150,2000)
basis(eval_points=eval_points).shape

(8, 2000, 1)

In [126]:
train(n_epochs=1)
accuracy=((torch.sum(torch.argmax(classifier(x_test),dim=1)==y_test_tensor)/x_test_tensor.shape[0])*100)
print("Précision moyenne =",((torch.mean(accuracy.float()))).detach().cpu().numpy(),"%")  

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_3556\2610319266.py:31: DeprecationWarning: The method 'evaluate' is deprecated. Please use the normal calling notation on the basis object instead.
  basis_fc=self.basis.evaluate(x.grid_points[0],derivative=0)[:,:,0]
  0%|          | 0/1 [00:00<?, ?it/s]

Précision moyenne = 94.40001 %


In [128]:
import gc
torch.cuda.empty_cache()
gc.collect()

53